In [ ]:
# Data Manipulations
import pandas as pd
import numpy as np

# Data Visualizations
import matplotlib.pyplot as plt
from matplotlib import rcParams
import statistics as s
import seaborn as sns

# apply some cool styling
plt.style.use("ggplot")
rcParams['figure.figsize'] = (12,  6)

# Data Analysis
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text
import getpass
import datetime

import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from scipy.stats import norm

import gspread

import mpl_toolkits.mplot3d 
from sklearn import datasets

import warnings
from functools import partial

from dateutil.relativedelta import relativedelta
import requests
import json
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)


In [23]:
redshift_endpoint = 'dw-staging.cr74dmrw5q8u.us-east-2.redshift.amazonaws.com'
redshift_user = 'tableau_analytics'
redshift_pass = 't4bID07!'
port = 5439
dbname = 'dev'

sql = """
WITH tl_poi
AS (
	SELECT 
	  tl.transaction_id
	  ,tl.transaction_line_id
	  ,tl.transaction_id || '.' || tl.transaction_line_id AS tl_poi_wid
		,tl.act_factory_finish_date
		,tl.confirmation_date
		,tl.act_loading_date
		,tl.carton_height
		,tl.carton_length
		,tl.carton_width
		,tl.est_cbm
		,tl.location_id
        ,tl.quantity_received_in_shipment
		,tl.shipment_received
	FROM landing_netsuite.transaction_lines AS tl
	INNER JOIN landing_netsuite.transactions AS t ON tl.transaction_id = t.transaction_id
	WHERE t.transaction_type = 'Purchase Order'
		AND tl.transaction_line_id <> 0
	)
	,non_std_ord
AS (
	SELECT fpoi.purchase_order_wid
		,CASE 
			WHEN dpo.magento_sales_order_id LIKE '9%'
				THEN 'non_std'
			WHEN dpo.magento_sales_order_id ISNULL
				THEN 'non_std'
			WHEN dpo.magento_sales_order_id LIKE 'H%'
				THEN 'non_std'
			WHEN dpo.magento_sales_order_id LIKE '1%'
				THEN 'miss_soi_wid'
			ELSE LEFT(dpo.magento_sales_order_id, 1)
			END AS std_index
	FROM datawarehouse.fact_purchase_order_items AS fpoi
	INNER JOIN datawarehouse.dim_date AS d ON fpoi.created_at_wid = d.wid
	INNER JOIN datawarehouse.dim_purchase_orders AS dpo ON fpoi.purchase_order_wid = dpo.wid
		WHERE d.year >= 2018
		AND fpoi.sales_order_items_wid ISNULL
	GROUP BY 1
		,2
	)
	, 
	max_delivery as (
	SELECT * FROM (
	SELECT dso.wid as so_wid 
  	, dds.domestic_shipment_id
		, dds.domestic_shipment_name
		, dds.STATUS
	  , fds.delivery_pod_wid
	  , d.date 
	  , row_number() over (
	    PARTITION BY dso.wid 
	    ORDER BY d.date DESC
	    ) as order_sequence
	  
	  FROM datawarehouse.dim_sales_orders as dso 
      LEFT JOIN datawarehouse.fact_domestic_shipments AS fds ON dso.wid = fds.order_wid
      LEFT JOIN datawarehouse.dim_domestic_shipments AS dds ON fds.domestic_shipment_wid = dds.wid
      LEFT JOIN datawarehouse.dim_date as d on cast(fds.delivery_pod_wid as date) = d.date
      
      
      ORDER BY dso.wid, order_sequence
  )
    where order_sequence = 1
  ) 

, poi AS (
  SELECT
    fpoi.purchase_order_item_wid AS wid,
    SUM(fpoi.quantity) AS po_units
  FROM
    datawarehouse.fact_purchase_order_items AS fpoi
  GROUP BY 1
),
fcs AS (
  SELECT fcsd.purchase_order_item_wid AS wid
    , SUM(fcsd.quantity) AS fc_units
  FROM datawarehouse.fact_container_shipment_details AS fcsd
  GROUP BY 1
), 
fcs_fil AS (
SELECT poi.wid
  , poi.po_units
  , fcs.fc_units
  , (poi.po_units = fcs.fc_units) AS t_f
  , fcs.fc_units - poi.po_units AS diff
FROM 
  poi 
  INNER JOIN fcs ON poi.wid = fcs.wid
WHERE t_f = False
ORDER BY diff
),
fcsd AS (
  SELECT 
    fcsd.purchase_order_item_wid
    , sum(fcsd.unit_cbm) AS ext_cbm
  FROM datawarehouse.fact_container_shipment_details AS fcsd
  WHERE fcsd.purchase_order_item_wid NOT IN (
                                            SELECT wid
                                            FROM fcs_fil
  )
  GROUP BY 1
)


	,com
AS (
	SELECT 
	-- Purchase Order Level Details
	  dso.magento_sales_order_id AS so_num_co
		,dpo.magento_sales_order_id AS so_num_ns
		,dpo.po_number
		,dpoi.po_line_number
		,dso.status AS SO_Status
		,dpo.status AS PO_Status
		,dpo.factory_status
		,dsoi.fulfillment_classification
		,CASE 
			WHEN non_std_ord.std_index ISNULL
				THEN 'std'
			ELSE non_std_ord.std_index
			END AS std_index
		,dpo.vendor
		,dsoi.bndl
		,tl_poi.location_id
		,loc.address
		,dso.email
		-- Dates for comparison
		,cast(fso.created_at_wid_ct AS DATE) AS so_date		
		,cast(fsoi.bndl_fabric_selected_wid AS DATE) AS bndl_date		
		,COALESCE(bndl_date, so_date) AS so_launch_date 
		,CAST(d.wid AS DATE) AS po_date		
		,CAST(fcs.actual_arrival_port_wid AS DATE) AS actual_arrival_port_date
		,CAST(fcs.actual_arrival_wh_wid AS DATE) AS actual_arrival_wh_date		
		,tl_poi.act_factory_finish_date
		,CAST(fcs.actual_departure_wid AS DATE) AS actual_departure_date
		,tl_poi.confirmation_date
		,tl_poi.act_loading_date
		,CAST(md.delivery_pod_wid AS DATE)
		, DATEDIFF ( 'days', so_launch_date,  CURRENT_DATE) AS date_diff
	 
  -- Product Level Detail
		,dsoi.sku
		,mh.division
		,mh.class
		,mh.sub_class
		,mh.micro_class
		,mh.nano_class
		,mh.pico_class
		,dp.collection
		,mh.mto_dropship
		
-- Component level measures		
		,dpoi.piece_description
		,fpoi.rate
		,fpoi.quantity AS comp_qty
		,fpoi.total_amount
		,fcsd.ext_cbm AS CBM
		,tl_poi.carton_height
		,tl_poi.carton_length
		,tl_poi.carton_width
		,tl_poi.est_cbm
		,tl_poi.quantity_received_in_shipment
		,tl_poi.shipment_received
-- Container Level Data
		,dpoi.container_name
		,dcs.container_tracking_id
	, fp.container_number AS flexport_container_number	
  , fp.departure_port_actual_departure_date
  , fp.arrival_port_actual_arrival_date
  , fp.arrival_port_actual_departure_date
  , fp.destination_actual_arrival_date
  -- Domestic Shipment Level Data		
		, md.domestic_shipment_id
		, md.domestic_shipment_name
		, md.STATUS
		,lds.order_complete_status_id
  , dso.shipping_zip
  
  , CASE

    WHEN so_status ='delivered' THEN 'Delivered'
    WHEN delivery_pod_wid > 0 THEN 'Issue: Delivery POD Date but SO_Status not 'delivered''

-- ELSEIF(ISNULL([Destination Actual Arrival Date])=false AND ISNULL([delivery_pod_wid])) THEN "At GTZ"
    WHEN (fp.destination_actual_arrival_date > 0  AND delivery_pod_wid IS NULL ) THEN 'At GTZ'
    WHEN (fcs.actual_arrival_wh_wid > 0 AND ISNULL([Flexport Container Number])=false AND delivery_pod_wid IS NULL ) THEN 'Issue: Netsuite WH Arrival Date but NO Flexport Arrival Date'
    WHEN (fcs.actual_arrival_wh_wid > 0 AND delivery_pod_wid IS NULL ) THEN 'At GTZ - Non-Flexport Container'
    
-- ELSEIF(ISNULL([Arrival Port Actual Departure Date])=false AND ISNULL([Flexport Container Number])=false AND ISNULL([Destination Actual Arrival Date])) THEN "At Drayage Partner"
  WHEN (fp.arrival_port_actual_departure_date > 0 AND (fp.destination_actual_arrival_date = '' OR fp.destination_actual_arrival_date IS NULL)) THEN 'At Drayage Partner'
  WHEN (fp.arrival_port_actual_departure_date > 0 AND fcs.actual_arrival_wh_wid IS NULL) THEN 'Issue: Netsuite '
  
-- ELSEIF(ISNULL([Arrival Port Actual Arrival Date])=false AND ISNULL([Flexport Container Number])=false AND ISNULL([Arrival Port Actual Departure Date])) THEN "At Arrival Port"
  WHEN (fp.arrival_port_actual_arrival_date > 0 AND (fp.arrival_port_actual_departure_date='' OR fp.arrival_port_actual_departure_date IS NULL)) THEN 'At Arrival Port'
  WHEN (fcs.actual_arrival_port_wid > 0) THEN 'At Arrival Port 2'

-- ELSEIF(ISNULL([Departure Port Actual Departure Date])=false AND ISNULL([Arrival Port Actual Arrival Date])) THEN "On Water"
  WHEN (fp.departure_port_actual_departure_date > 0 AND (fp.arrival_port_actual_arrival_date='' OR fp.arrival_port_actual_arrival_date IS NULL) ) THEN 'On Water'
  WHEN (fcs.actual_departure_wid > 0 AND fcs.actual_arrival_port_wid IS NULL) THEN 'On Water'

-- ELSEIF(ISNULL([Actual Factory Finish Date])=false AND ISNULL([Departure Port Actual Departure Date])) THEN "PO Complete"
  WHEN (tl_poi.act_factory_finish_date > 0 AND (fp.departure_port_actual_departure_date='' OR fp.departure_port_actual_departure_date IS NULL)) THEN 'PO Complete'
  WHEN (tl_poi.act_factory_finish_date > 0 AND fcs.actual_departure_wid IS NULL) THEN 'PO Complete'

-- ELSEIF(ISNULL([PO Confirmation Date])=false AND [factory_status]<>"Closed" AND ISNULL([Actual Factory Finish Date])) THEN "PO Confirmed"
  WHEN (tl_poi.confirmation_date > 0 AND dpo.factory_status <>'Closed' AND tl_poi.act_factory_finish_date IS NULL) THEN 'PO Confirmed'

-- ELSEIF (ISNULL([PO Confirmation Date]) AND [factory_status]="Processing") THEN "PO Created"
  WHEN (tl_poi.confirmation_date IS NULL AND dpo.factory_status <>'Closed' AND dpo.factory_status <>'Cancellation') THEN 'PO Created'
  
-- END
  END AS carton_status
    
	FROM datawarehouse.fact_purchase_order_items AS fpoi
	INNER JOIN datawarehouse.dim_purchase_order_items AS dpoi ON fpoi.purchase_order_item_wid = dpoi.wid
	INNER JOIN datawarehouse.fact_purchase_orders AS fpo ON fpoi.purchase_order_wid = fpo.purchase_order_wid
	INNER JOIN datawarehouse.dim_purchase_orders AS dpo ON fpo.purchase_order_wid = dpo.wid
	INNER JOIN datawarehouse.dim_date AS d ON fpo.po_created_date_wid = d.wid
	LEFT JOIN datawarehouse.dim_sales_orders AS dso ON fpo.order_wid = dso.wid
	LEFT JOIN datawarehouse.dim_sales_order_items AS dsoi ON fpoi.sales_order_items_wid = dsoi.wid
	LEFT JOIN datawarehouse.fact_sales_orders AS fso ON fpo.order_wid = fso.order_wid
	LEFT JOIN datawarehouse.fact_sales_order_items AS fsoi ON fpoi.sales_order_items_wid = fsoi.sales_order_items_wid
	LEFT JOIN datawarehouse.dim_products AS dp ON fsoi.product_wid = dp.wid
	LEFT JOIN datawarehouse.dim_container_shipments AS dcs ON dcs.container_name = dpoi.container_name
	LEFT JOIN datawarehouse.fact_container_shipments AS fcs ON fcs.container_shipment_wid = dcs.wid
	LEFT JOIN max_delivery AS md ON dso.wid = md.so_wid
	LEFT JOIN fcsd ON dpoi.wid = fcsd.purchase_order_item_wid
	--LEFT JOIN datawarehouse.dim_domestic_shipments AS dds ON fds.domestic_shipment_wid = dds.wid
	LEFT JOIN landing_netsuite.domestic_shipment AS lds ON lds.domestic_shipment_id = md.domestic_shipment_id
	LEFT JOIN non_std_ord ON fpoi.purchase_order_wid = non_std_ord.purchase_order_wid
	LEFT JOIN landing_google_sheets.merchandise_hierarchy AS mh ON dsoi.sku = mh.clean_sku
	LEFT JOIN tl_poi ON (dpo.netsuite_purchase_order_id = tl_poi.transaction_id AND dpoi.po_line_number = tl_poi.transaction_line_id)
	LEFT JOIN landing_netsuite.locations AS loc ON tl_poi.location_id = loc.location_id
	LEFT JOIN analysts.flexport as fp ON dpoi.container_name = fp.container_number
	WHERE d.year >= 2020

	--WHERE d.year = 2022 AND d.month = 2 --  7,635 CNT / $2,578,460.36 (2,611,175.97)
	--WHERE d.year >= 2020                -- 210,976 / $65,050,252.42
	ORDER BY so_num_co
		,dpo.po_number
		,dpoi.po_line_number
	)

SELECT * FROM com

"""

In [24]:
from sqlalchemy import create_engine
from sqlalchemy import text
engine_string = "postgresql+psycopg2://%s:%s@%s:%d/%s" \
% (redshift_user, redshift_pass, redshift_endpoint, int(port), dbname)
engine = create_engine(engine_string)

In [25]:
df = pd.read_sql_query(text(sql), engine)

In [26]:
df.head()

,so_num_co,so_num_ns,po_number,po_line_number,so_status,po_status,factory_status,fulfillment_classification,std_index,vendor,bndl,location_id,address,email,so_date,bndl_date,so_launch_date,po_date,actual_arrival_port_date,actual_arrival_wh_date,act_factory_finish_date,actual_departure_date,confirmation_date,act_loading_date,delivery_pod_wid,date_diff,sku,division,class,sub_class,micro_class,nano_class,pico_class,collection,mto_dropship,piece_description,rate,comp_qty,total_amount,cbm,carton_height,carton_length,carton_width,est_cbm,quantity_received_in_shipment,shipment_received,container_name,container_tracking_id,flexport_container_number,departure_port_actual_departure_date,arrival_port_actual_arrival_date,arrival_port_actual_departure_date,destination_actual_arrival_date,domestic_shipment_id,domestic_shipment_name,status,order_complete_status_id,shipping_zip,carton_status
0,100047322,100047322,HO47322ID,1.0,delivered,Fully Billed,Closed,None,std,Holly Wood,True,8.0,"Ontario, CA\r\nLos Angeles CA 90001\r\nUnited ...",tgnovy@gmail.com,2019-07-23,2020-12-08,2020-12-08,2020-02-12,None,None,2020-03-17 00:00:00+00:00,None,2020-02-25 00:00:00+00:00,2020-04-01 00:00:00+00:00,None,681.0,JASN.FABRIC.SOFA.APRTMT,Furniture,Sofas,Loveseat,Loveseat,Loveseat,Loveseat,Jason,MTO,Sofa,565.51,1.0,565.51,NaN,38.0,77.0,40.0,1.89,1.0,2020-04-05 00:00:00+00:00,RFCU4018385,NaN,RFCU4018385,2020-04-04,2020-04-20,2020-04-24,2020-04-24,NaN,None,None,NaN,60190,Delivered
1,100047454,100047454,HO47454ID,1.0,shipped_la,Fully Billed,Closed,None,std,Holly Wood,True,8.0,"Ontario, CA\r\nLos Angeles CA 90001\r\nUnited ...",wolfahoward@gmail.com,2019-07-26,2021-02-12,2021-02-12,2020-07-23,None,None,2020-08-18 00:00:00+00:00,None,2020-07-24 00:00:00+00:00,2020-08-23 00:00:00+00:00,None,615.0,JASN.LEATHR.CHAR.ACCENT,Furniture,Chairs,Accent Regular,Accent Regular,Accent Regular,Accent Regular,Jason,MTO,Chair,703.42,1.0,703.42,1.01,NaN,NaN,NaN,1.01,1.0,2020-08-21 00:00:00+00:00,CMAU5813306,20701.0,CMAU5813306,2020-09-18,2020-10-03,2020-10-08,2020-10-09,16118.0,9851501,Update Status,NaN,63103,At GTZ
2,100047616,100047616,SA47616ID,1.0,delivered,Fully Billed,Closed,None,std,Samanni,True,8.0,"Ontario, CA\r\nLos Angeles CA 90001\r\nUnited ...",prosenheimer@gmail.com,2019-07-30,2020-12-08,2020-12-08,2020-01-07,None,None,2020-02-27 00:00:00+00:00,None,2020-01-09 00:00:00+00:00,2020-02-28 00:00:00+00:00,None,681.0,SLON.FABRIC.SECT.CSLEFT,Furniture,Sectionals,Chaise,Left,Left,Left,Sloan,MTO,1-Arm Sofa,218.97,1.0,218.97,NaN,27.0,64.0,39.0,1.11,1.0,2020-02-28 00:00:00+00:00,EMCU9650088,NaN,EMCU9650088,2020-03-08,2020-03-22,2020-03-25,2020-03-25,NaN,None,None,NaN,60622,Delivered
3,100047616,100047616,SA47616ID,2.0,delivered,Fully Billed,Closed,None,std,Samanni,True,8.0,"Ontario, CA\r\nLos Angeles CA 90001\r\nUnited ...",prosenheimer@gmail.com,2019-07-30,2020-12-08,2020-12-08,2020-01-07,None,None,2020-02-27 00:00:00+00:00,None,2020-01-09 00:00:00+00:00,2020-02-28 00:00:00+00:00,None,681.0,SLON.FABRIC.SECT.CSLEFT,Furniture,Sectionals,Chaise,Left,Left,Left,Sloan,MTO,Left Facing Chaise,207.35,1.0,207.35,NaN,27.0,39.0,66.0,1.14,1.0,2020-02-28 00:00:00+00:00,EMCU9650088,NaN,EMCU9650088,2020-03-08,2020-03-22,2020-03-25,2020-03-25,NaN,None,None,NaN,60622,Delivered
4,100047681,100047681,SA047681ID,1.0,in_production,Fully Billed,Closed,None,std,Samanni,True,8.0,"Ontario, CA\r\nLos Angeles CA 90001\r\nUnited ...",maddie1031@gmail.com,2019-07-31,2022-05-06,2022-05-06,2022-05-09,2022-08-09,None,2022-07-14 00:00:00+00:00,2022-07-25,2022-05-18 00:00:00+00:00,2022-07-19 00:00:00+00:00,None,167.0,CAIT.FABRIC.CHAR.ACCENT,Furniture,Chairs,Accent Regular,Accent Regular,Accent Regular,Accent Regular,Caitlin,MTO,Chair,273.15,1.0,273.15,0.74,29.0,42.0,40.0,0.85,1.0,2022-07-19 00:00:00+00:00,TLLU4025976,276802.0,TLLU4025976,2022-07-26,2022-08-09,2022-08-17,,186278.0,186278,Pending Transmission,2.0,60613,At Drayage Partner


In [27]:
df.shape

(219992, 59)

In [28]:
df.to_clipboard(index=False)

In [29]:
df.carton_status.value_counts(dropna=False).sort_values(ascending=False)

Delivered             150380
At GTZ                 15967
PO Confirmed           13388
At Drayage Partner     11823
PO Created              9967
PO Complete             8440
At Arrival Port         5155
None                    4521
On Water                 351
Name: carton_status, dtype: int64

In [47]:
#Per Meeting with Sean, what is the best way to indicate an "open" order from Netsuite info? 

# MTO Only
#SOI SO Line item Qty - field "Fulfilled" = 0 then "Delivered" (SOI "Fulfilled" Field needs to be added to DW FSOI structure)
#SOI "Closed" field = "Yes" then status as closed/cancelled and do not include in Open Order Report (SOI "Closed" field needs to be added to DW FSOI structure)
###   Closed = yes can mean closed before factory made the item in which case factory status = cancelled, OR closed = yes but factory already started prdcution so factory status <> cancelled meand open order but not headed to customer (will be re-directed to Quickship or storage)
# Likley just filter these out: SO# that start with 9 indicate as "Studio" (does this follow the same Item fulfillment "Yes" proicess?? if not, what indicates the item has been delivered to GTZ / is in storage?
# OI will follow the same "Fulfilled" and "Closed" field  
#????? Indicator for studio/OI Would This be "9000%" SO?







df = df[df['carton_status'] != 'Delivered']
df = df[df['std_index'] != 'miss_soi_wid']
df = df[df['std_index'] != 'non_std']
df = df[df['mto_dropship'] != 'Dropship']
df = df[df['factory_status'] != 'Cancellation']
df = df[df['so_status'] != 'cancellation']
df = df[df['vendor'] != 'JL Jonathan Louis']
df = df[df['container_name'] != 'CAAU5853556']
df = df[df['po_number'] != 'SA69446ID']
df = df[df['po_number'] != 'SA96253ID']
df = df[df['po_number'] != 'HO109971ID']
df = df[df['po_number'] != 'HO113743ID']
df = df[df['po_number'] != 'SH119071ID']
df = df[df['po_number'] != 'SH119788ID']
df = df[df['po_number'] != 'SH113928ID']
df = df[df['po_number'] != 'SH115711ID']
df = df[df['po_number'] != '100073941']
df = df[df['po_number'] != 'SA63706ID-1']


df['destination_actual_arrival_date'] = pd.to_datetime(df['destination_actual_arrival_date'])
df = df[(df['destination_actual_arrival_date'] >= datetime.datetime(2022,1,1)) | (df['destination_actual_arrival_date'].isnull())]

df['so_date'] = pd.to_datetime(df['so_date'])
df = df[(df['so_date'] >= datetime.datetime(2022,1,1)) | (df['so_date'].isnull())]

df['so_launch_date'] = pd.to_datetime(df['so_launch_date'])
df = df[(df['so_launch_date'] >= datetime.datetime(2022,1,1)) | (df['so_launch_date'].isnull())]

df['arrival_port_actual_arrival_date'] = pd.to_datetime(df['arrival_port_actual_arrival_date'])
df = df[(df['so_launch_date'] >= datetime.datetime(2022,5,29)) | (df['arrival_port_actual_arrival_date'].isnull())]



KeyError: 'std_index'

In [31]:
df['confirmation_date'] =pd.to_datetime(df['confirmation_date'].dt.date)
df['act_factory_finish_date'] =pd.to_datetime(df['act_factory_finish_date'].dt.date)

In [32]:
df.shape

(42785, 59)

In [33]:
df.std_index.value_counts(dropna=False).sort_values(ascending=False)


std    42785
Name: std_index, dtype: int64

In [34]:
df.carton_status.value_counts(dropna=False).sort_values(ascending=False)

At Drayage Partner    11568
PO Confirmed          10544
PO Complete            7266
PO Created             6744
At Arrival Port        4851
At GTZ                 1441
On Water                341
None                     30
Name: carton_status, dtype: int64

In [35]:
df.head()

,so_num_co,so_num_ns,po_number,po_line_number,so_status,po_status,factory_status,fulfillment_classification,std_index,vendor,bndl,location_id,address,email,so_date,bndl_date,so_launch_date,po_date,actual_arrival_port_date,actual_arrival_wh_date,act_factory_finish_date,actual_departure_date,confirmation_date,act_loading_date,delivery_pod_wid,date_diff,sku,division,class,sub_class,micro_class,nano_class,pico_class,collection,mto_dropship,piece_description,rate,comp_qty,total_amount,cbm,carton_height,carton_length,carton_width,est_cbm,quantity_received_in_shipment,shipment_received,container_name,container_tracking_id,flexport_container_number,departure_port_actual_departure_date,arrival_port_actual_arrival_date,arrival_port_actual_departure_date,destination_actual_arrival_date,domestic_shipment_id,domestic_shipment_name,status,order_complete_status_id,shipping_zip,carton_status
40402,100078776,100078776,HO078776ID,1.0,in_production,Fully Billed,Closed,None,std,Holly Wood,True,8.0,"Ontario, CA\r\nLos Angeles CA 90001\r\nUnited ...",arianna@roaringlionfarm.com,2020-09-30,2022-05-04,2022-05-04,2022-05-04,2022-08-02,None,2022-06-28,2022-07-19,2022-05-06,2022-07-14 00:00:00+00:00,None,169.0,SLON.FABRIC.CHAR.ACCENT,Furniture,Chairs,Accent Regular,Accent Regular,Accent Regular,Accent Regular,Sloan,MTO,Chair,173.93,1.0,173.93,1.74,38.0,54.0,52.0,0.81,1.0,2022-07-18 00:00:00+00:00,GCXU5005538,273002.0,GCXU5005538,2022-07-20,2022-08-02,,NaT,181197.0,181197,Pending Transmission,2.0,04676,At Arrival Port
40403,100078776,100078776,HO078776ID,2.0,in_production,Fully Billed,Closed,None,std,Holly Wood,True,8.0,"Ontario, CA\r\nLos Angeles CA 90001\r\nUnited ...",arianna@roaringlionfarm.com,2020-09-30,2022-05-04,2022-05-04,2022-05-04,2022-08-02,None,2022-06-28,2022-07-19,2022-05-06,2022-07-14 00:00:00+00:00,None,169.0,SLON.FABRIC.OTTO.STNDRD,Furniture,Ottomans,Ottomans,Standard,Standard,Standard,Sloan,MTO,Ottoman,107.42,1.0,107.42,0.34,16.0,42.0,31.0,0.33,1.0,2022-07-18 00:00:00+00:00,GCXU5005538,273002.0,GCXU5005538,2022-07-20,2022-08-02,,NaT,181197.0,181197,Pending Transmission,2.0,04676,At Arrival Port
52490,100085173,100085173,SA085173ID,1.0,in_production,Pending Receipt,Received,Made To Order,std,Samanni,True,8.0,"Ontario, CA\r\nLos Angeles CA 90001\r\nUnited ...",erinyoungman@gmail.com,2020-11-28,2022-08-11,2022-08-11,2022-08-11,None,None,NaT,None,2022-08-29,NaT,None,70.0,CHES.FABRIC.SOFA.STNDRD,Furniture,Sofas,2 Seat,2 Seat,2 Seat,2 Seat,Ms. Chesterfield,MTO,Sofa,441.61,1.0,441.61,NaN,NaN,NaN,NaN,1.15,0.0,NaT,None,NaN,None,None,None,None,NaT,NaN,None,None,NaN,13837,PO Confirmed
52491,100085173,100085173,SA085173ID,2.0,in_production,Pending Receipt,Received,Made To Order,std,Samanni,True,8.0,"Ontario, CA\r\nLos Angeles CA 90001\r\nUnited ...",erinyoungman@gmail.com,2020-11-28,2022-08-11,2022-08-11,2022-08-11,None,None,NaT,None,2022-08-29,NaT,None,70.0,CHES.FABRIC.SOFA.LOVESEAT,Furniture,Sofas,Loveseat,Loveseat,Loveseat,Loveseat,Ms. Chesterfield,MTO,Sofa,441.61,1.0,441.61,NaN,NaN,NaN,NaN,0.92,0.0,NaT,None,NaN,None,None,None,None,NaT,NaN,None,None,NaN,13837,PO Confirmed
55951,100086974,100086974,SA086974ID,1.0,in_production,Fully Billed,Closed,Made To Order,std,Samanni,True,8.0,"Ontario, CA\r\nLos Angeles CA 90001\r\nUnited ...",sarahhaslip@gmail.com,2020-12-02,2022-05-16,2022-05-16,2022-05-16,2022-08-14,None,2022-06-30,2022-07-14,2022-05-27,2022-07-01 00:00:00+00:00,None,157.0,CHLY.FABRIC.SECT.CORNER,Furniture,Sectionals,Corner,Corner,Corner,Corner,Charly,MTO,1-Arm Sofa(Left) - Standard (37”),339.51,1.0,339.51,1.42,29.0,74.0,40.0,1.42,1.0,2022-07-07 00:00:00+00:00,CMAU9552411,266502.0,CMAU9552411,2022-07-15,2022-08-14,,NaT,177259.0,177259,Pending Transmission,2.0,85142,At Arrival Port


In [36]:
print("Column headers from list(df.columns.values):",
      list(df.columns.values))

Column headers from list(df.columns.values): ['so_num_co', 'so_num_ns', 'po_number', 'po_line_number', 'so_status', 'po_status', 'factory_status', 'fulfillment_classification', 'std_index', 'vendor', 'bndl', 'location_id', 'address', 'email', 'so_date', 'bndl_date', 'so_launch_date', 'po_date', 'actual_arrival_port_date', 'actual_arrival_wh_date', 'act_factory_finish_date', 'actual_departure_date', 'confirmation_date', 'act_loading_date', 'delivery_pod_wid', 'date_diff', 'sku', 'division', 'class', 'sub_class', 'micro_class', 'nano_class', 'pico_class', 'collection', 'mto_dropship', 'piece_description', 'rate', 'comp_qty', 'total_amount', 'cbm', 'carton_height', 'carton_length', 'carton_width', 'est_cbm', 'quantity_received_in_shipment', 'shipment_received', 'container_name', 'container_tracking_id', 'flexport_container_number', 'departure_port_actual_departure_date', 'arrival_port_actual_arrival_date', 'arrival_port_actual_departure_date', 'destination_actual_arrival_date', 'domestic_

In [37]:
# Drop unneeded columns
df = df.drop([
 'so_status'
, 'po_status'
, 'so_num_co'
, 'factory_status'
, 'std_index'
, 'bndl'
, 'location_id'
, 'address'
, 'so_date'
, 'bndl_date'
, 'po_date'
, 'act_loading_date'
, 'delivery_pod_wid'
, 'date_diff'
, 'division'
, 'class'
, 'sub_class'
, 'micro_class'
, 'nano_class'
, 'pico_class'
, 'collection'
, 'mto_dropship'
, 'rate'
, 'carton_height'
, 'carton_length'
, 'carton_width'
, 'quantity_received_in_shipment'
, 'shipment_received'
, 'container_tracking_id'
, 'domestic_shipment_id'
, 'domestic_shipment_name'
, 'status'
, 'order_complete_status_id'
, 'fulfillment_classification'
]
, axis=1)

In [38]:
df.head()

,so_num_co,so_num_ns,po_number,po_line_number,fulfillment_classification,vendor,email,so_launch_date,actual_arrival_port_date,actual_arrival_wh_date,act_factory_finish_date,actual_departure_date,confirmation_date,sku,piece_description,comp_qty,total_amount,cbm,est_cbm,container_name,flexport_container_number,departure_port_actual_departure_date,arrival_port_actual_arrival_date,arrival_port_actual_departure_date,destination_actual_arrival_date,shipping_zip,carton_status
40402,100078776,100078776,HO078776ID,1.0,None,Holly Wood,arianna@roaringlionfarm.com,2022-05-04,2022-08-02,None,2022-06-28,2022-07-19,2022-05-06,SLON.FABRIC.CHAR.ACCENT,Chair,1.0,173.93,1.74,0.81,GCXU5005538,GCXU5005538,2022-07-20,2022-08-02,,NaT,04676,At Arrival Port
40403,100078776,100078776,HO078776ID,2.0,None,Holly Wood,arianna@roaringlionfarm.com,2022-05-04,2022-08-02,None,2022-06-28,2022-07-19,2022-05-06,SLON.FABRIC.OTTO.STNDRD,Ottoman,1.0,107.42,0.34,0.33,GCXU5005538,GCXU5005538,2022-07-20,2022-08-02,,NaT,04676,At Arrival Port
52490,100085173,100085173,SA085173ID,1.0,Made To Order,Samanni,erinyoungman@gmail.com,2022-08-11,None,None,NaT,None,2022-08-29,CHES.FABRIC.SOFA.STNDRD,Sofa,1.0,441.61,NaN,1.15,None,None,None,None,None,NaT,13837,PO Confirmed
52491,100085173,100085173,SA085173ID,2.0,Made To Order,Samanni,erinyoungman@gmail.com,2022-08-11,None,None,NaT,None,2022-08-29,CHES.FABRIC.SOFA.LOVESEAT,Sofa,1.0,441.61,NaN,0.92,None,None,None,None,None,NaT,13837,PO Confirmed
55951,100086974,100086974,SA086974ID,1.0,Made To Order,Samanni,sarahhaslip@gmail.com,2022-05-16,2022-08-14,None,2022-06-30,2022-07-14,2022-05-27,CHLY.FABRIC.SECT.CORNER,1-Arm Sofa(Left) - Standard (37”),1.0,339.51,1.42,1.42,CMAU9552411,CMAU9552411,2022-07-15,2022-08-14,,NaT,85142,At Arrival Port


In [39]:
# Reorder Columns
df = df[[
'so_launch_date'
, 'so_num_ns'
, 'po_number'
, 'po_line_number'
, 'sku'
, 'piece_description'    
, 'comp_qty'
, 'total_amount'
, 'carton_status'
, 'container_name'
, 'flexport_container_number'
, 'vendor'
, 'shipping_zip'
, 'email'
, 'est_cbm'
, 'cbm'
, 'confirmation_date'
, 'act_factory_finish_date'
, 'departure_port_actual_departure_date'
, 'actual_departure_date'
, 'arrival_port_actual_arrival_date'
, 'actual_arrival_port_date'
, 'arrival_port_actual_departure_date'
, 'destination_actual_arrival_date'
, 'actual_arrival_wh_date'
]
]

In [44]:
sorted_df=df.sort_values(by = ['destination_actual_arrival_date','actual_arrival_wh_date', 'arrival_port_actual_departure_date','arrival_port_actual_arrival_date','actual_arrival_port_date','departure_port_actual_departure_date','actual_departure_date','act_factory_finish_date','so_launch_date'])

In [41]:
df.head()

,so_launch_date,so_num_co,so_num_ns,po_number,po_line_number,sku,piece_description,comp_qty,total_amount,carton_status,container_name,flexport_container_number,vendor,shipping_zip,email,fulfillment_classification,est_cbm,cbm,confirmation_date,act_factory_finish_date,departure_port_actual_departure_date,actual_departure_date,arrival_port_actual_arrival_date,actual_arrival_port_date,arrival_port_actual_departure_date,destination_actual_arrival_date,actual_arrival_wh_date
40402,2022-05-04,100078776,100078776,HO078776ID,1.0,SLON.FABRIC.CHAR.ACCENT,Chair,1.0,173.93,At Arrival Port,GCXU5005538,GCXU5005538,Holly Wood,04676,arianna@roaringlionfarm.com,None,0.81,1.74,2022-05-06,2022-06-28,2022-07-20,2022-07-19,2022-08-02,2022-08-02,,NaT,None
40403,2022-05-04,100078776,100078776,HO078776ID,2.0,SLON.FABRIC.OTTO.STNDRD,Ottoman,1.0,107.42,At Arrival Port,GCXU5005538,GCXU5005538,Holly Wood,04676,arianna@roaringlionfarm.com,None,0.33,0.34,2022-05-06,2022-06-28,2022-07-20,2022-07-19,2022-08-02,2022-08-02,,NaT,None
52490,2022-08-11,100085173,100085173,SA085173ID,1.0,CHES.FABRIC.SOFA.STNDRD,Sofa,1.0,441.61,PO Confirmed,None,None,Samanni,13837,erinyoungman@gmail.com,Made To Order,1.15,NaN,2022-08-29,NaT,None,None,None,None,None,NaT,None
52491,2022-08-11,100085173,100085173,SA085173ID,2.0,CHES.FABRIC.SOFA.LOVESEAT,Sofa,1.0,441.61,PO Confirmed,None,None,Samanni,13837,erinyoungman@gmail.com,Made To Order,0.92,NaN,2022-08-29,NaT,None,None,None,None,None,NaT,None
55951,2022-05-16,100086974,100086974,SA086974ID,1.0,CHLY.FABRIC.SECT.CORNER,1-Arm Sofa(Left) - Standard (37”),1.0,339.51,At Arrival Port,CMAU9552411,CMAU9552411,Samanni,85142,sarahhaslip@gmail.com,Made To Order,1.42,1.42,2022-05-27,2022-06-30,2022-07-15,2022-07-14,2022-08-14,2022-08-14,,NaT,None


In [46]:
sorted_df.to_csv(r'C:\Users\traci\Downloads\Exceptions_List.csv',index = False)